<a href="https://colab.research.google.com/github/shyDaniel/PM2.5_prediction_regression/blob/master/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **PREDICTING PM2.5 WITH LINEAR REGRESSION, TECHINIQUES INCLUDING FEATURE ENGINEERING, CROSS-VALIDATION, NORMALIZATION, AND ADAGRAD GRADIENT OPTIMIZATION.**

Kaggle: https://www.kaggle.com/c/ml2020spring-hw1/overview

Hanyu Song 03/19/2020

In [2]:
import pandas as pd
import numpy as np
from google.colab import drive
# import train data from google drive
! gdown --id '1VR_MKDGwhexEThy4VEZudoN0zKgNRpys'
train = pd.read_csv('./train.csv', encoding = 'big5')

Downloading...
From: https://drive.google.com/uc?id=1VR_MKDGwhexEThy4VEZudoN0zKgNRpys
To: /content/train.csv
100% 466k/466k [00:00<00:00, 67.4MB/s]


## **FEATURE ENGINEERING FOR TRAINING DATASET**



In [0]:
# delete unnecessary columns, fill in NR values, change to numpy form
train = train.iloc[:, 3:]
train[train == 'NR'] = 0
train_data = train.to_numpy()

In [0]:
# now split train data into 12 (months) blocks in which each block contains 18 (features )*480 (24 hours * 20 days) info
monthly_train = {}
for month in range(12):
  temp = np.empty((18, 480))
  for day in range(20):
    temp[:, day*24 : (day + 1)*24] = train_data[18 * (20*month + day) : 18 * (20 * month + day + 1), :]
  monthly_train[month] = temp;

In [0]:
# we will use previous 9 hours to data to predict the next, which is the 10th hour of pm2.5 level. 
# Therefore, out of the 480 hours we have for a given month, we have 471 sets of data (480 - 9) that can be used for training.
# y shape: (12*471) * 1
# x shape: (12*471) * (9*18)
# weight shape: (9*18) * 1
# y = x*w

x = np.empty((12*471, 9*18), dtype = float)
y = np.empty((12*471, 1), dtype = float)

for month in range(12):
  for day in range(20):
    for hour in range(24):
      if day == 0 and hour < 9:
          continue
      y[month * 471 + day * 24 + hour - 9, 0] = monthly_train[month][9, day * 24 + hour] #PM2.5 is on row 9

for month in range(12):
  for day in range(20):
    for hour in range(24):
      if day == 19 and hour > 14: 
        continue
      x[month * 471 + day * 24 + hour, :] = monthly_train[month][:, day * 24 + hour : day * 24 + hour + 9].reshape(1, -1)

## **NORMALIZATION**

In [0]:
# for each of the 18 features, compute their mean and std 
# then use newdata = (data - mean) /std to update x
mean_x = np.mean(x, axis = 0) #18 * 9 
std_x = np.std(x, axis = 0) #18 * 9 
normed_x = np.empty(x.shape, dtype = float)
for i in range(len(x)): #12 * 471
    for j in range(len(x[0])): #18 * 9 
        if std_x[j] != 0:
            normed_x[i][j] = (x[i][j] - mean_x[j]) / std_x[j]

## **CROSS-VALIDATION**

In [0]:
# This is an illustration of how we would split dataset into 4:1 train and validate
# x_train_set = x[: math.floor(len(normed_x) * 0.8), :]
# y_train_set = y[: math.floor(len(y) * 0.8), :]
# x_validation = x[math.floor(len(normed_x) * 0.8): , :]
# y_validation = y[math.floor(len(y) * 0.8): , :]

In [0]:
import math
import random as rand

dim = 9 * 18 + 1 # 9 * 18 features with 1 more constant
learning_rate = 100
iter_time = 10000
adagrad = np.zeros([dim, 1])
eps = 0.0000000001
sum_loss = 0
k_fold = 5

# Do 5-fold cross-validation on the normed_x and y
for i in range(k_fold):
  w = np.zeros((dim, 1))
  # create training and validating data for each iteration
  x_train_set = np.concatenate((normed_x[: math.floor(len(normed_x) * (0.2*i)), :], normed_x[math.floor(len(normed_x) * 0.2*(i + 1)) :, :]))
  x_validate_set = normed_x[math.floor(len(normed_x) * (0.2*i)) : math.floor(len(normed_x) * 0.2* (i + 1))]
  y_train_set = np.concatenate((y[:math.floor(len(y) * (0.2*i)), :], y[math.floor(len(y) * 0.2*(i + 1)):, :]))
  y_validate_set = y[math.floor(len(y) * (0.2*i)) : math.floor(len(y) * 0.2* (i + 1))]
 
  # x train plus one row of constant (to test the weight for the constant term)
  temp_x = np.concatenate((np.ones((x_train_set.shape[0], 1)), x_train_set), axis = 1)
  x_validate_set = np.concatenate((np.ones((x_validate_set.shape[0], 1)), x_validate_set), axis = 1)
  gradient = np.zeros((dim, 1))

  for t in range(iter_time):
    loss = np.sqrt(np.sum(np.power(np.dot(temp_x, w) - y_train_set, 2))/471/12)#rmse
    if(t%1000==0):
        print("  " + str(t) + ":" + str(loss))
    # gradient descent
    gradient = 2 * np.dot(temp_x.transpose(), np.dot(temp_x, w) - y_train_set) #dim*1

    # if (gradient){
    #     break;
    # }

    #adagrad gradient optimization
    adagrad += gradient ** 2
    w = w - learning_rate * gradient / np.sqrt(adagrad + eps)
  
  final_w += w
  print("for the ", i,"th time, the parameter is roughly ", w[rand.randrange(1,18),0])
  valid_loss = np.sqrt(np.sum(np.power(np.dot(x_validate_set, w) - y_validate_set, 2))/471/12)
  print("the loss for the ", i,"th time validate is ", valid_loss)
  sum_loss += valid_loss

print('average loss for the 5-fold validation is: ', sum_loss/k_fold)

## **TRAINING**

In [8]:
dim = 9 * 18 + 1 # 9 * 18 features with 1 more constant
learning_rate = 0.3
# iter_time = 5000
t = 0
oldloss = 1
newloss = 1
adagrad = np.zeros([dim, 1])
eps = 0.0000000001
w = np.random.rand(dim ,1)
normed_x1 = np.concatenate((np.ones([12 * 471, 1]), normed_x), axis = 1)

while True:
    oldloss = newloss
    loss = np.sqrt(np.sum(np.power(np.dot(normed_x1, w) - y, 2))/471/12)#rmse
    newloss = loss
    if (abs((newloss - oldloss)/oldloss) < 0.0000008):
      break;
    if(t%1000==0):
        print(str(t) + ":" + str(loss))
    gradient = 2 * np.dot(normed_x1.transpose(), np.dot(normed_x1, w) - y) #dim*1
    adagrad += gradient ** 2
    w = w - learning_rate * gradient / np.sqrt(adagrad + eps)
    t = t + 1
np.save('model_weights.npy', w)
w

0:30.347609310640646
1000:8.734215453347323
2000:6.443463205915888
3000:5.867558261495341
4000:5.729875927482143
5000:5.696009969243451
6000:5.68680290754431


array([[ 2.12490347e+01],
       [ 3.36594720e-01],
       [-5.46746148e-01],
       [ 5.69629374e-01],
       [-1.37026123e+00],
       [ 1.53012825e-01],
       [ 7.18439260e-01],
       [-4.53795246e-01],
       [-1.29188340e+00],
       [ 1.78546046e+00],
       [-3.29298805e-01],
       [ 1.41828540e-01],
       [ 6.35711811e-02],
       [ 1.10033269e-01],
       [-2.86264263e-02],
       [-3.90982434e-02],
       [-1.65993242e-01],
       [ 1.64986734e-01],
       [ 5.24532911e-01],
       [ 5.38425432e-02],
       [-2.28782655e-02],
       [ 6.32837060e-02],
       [-1.54997641e-01],
       [ 1.52672583e-01],
       [-2.27534686e-02],
       [-1.66620253e-01],
       [ 8.30548944e-02],
       [ 4.06962639e-01],
       [-3.21995170e-01],
       [ 3.67557006e-01],
       [-2.57704067e-01],
       [ 3.11956117e-01],
       [ 2.89220524e-01],
       [-5.17354652e-01],
       [ 1.17305777e-01],
       [ 1.51924401e-01],
       [ 1.06038609e-01],
       [-5.51793133e-03],
       [ 1.3

## **FEATURE ENGINEERING FOR TESTING DATASET**

In [9]:
! gdown --id '1JnF9biNzFqx5_9RKzCKPKgHPggtDHhue'
test = pd.read_csv('./test.csv', encoding = 'big5')
test = test.iloc[:, 2:]
test[test == 'NR'] = 0
test = test.to_numpy()

Downloading...
From: https://drive.google.com/uc?id=1JnF9biNzFqx5_9RKzCKPKgHPggtDHhue
To: /content/test.csv
100% 197k/197k [00:00<00:00, 68.6MB/s]


In [0]:
# noticed that test data lacked the first row of data, which is AMB_TEMP, manually fill them up
add = np.empty((1,9))
add[0] = [21, 21, 20, 20, 19, 19, 19, 18, 17]
test = np.concatenate((add,test), axis = 0)
test_x = np.empty([240, 18*9], dtype = float)
for i in range(240):
    test_x[i, :] = test[18 * i: 18* (i + 1), :].reshape(1, -1)

## **NORMALIZATION FOR TESTING DATASET**

In [0]:
for i in range(len(test_x)):
    for j in range(len(test_x[0])):
        if std_x[j] != 0:
            test_x[i][j] = (test_x[i][j] - mean_x[j]) / std_x[j]
normed_test_x = np.concatenate((np.ones([240, 1]), test_x), axis = 1)

## **PREDICTION**

In [0]:
test_w = np.load('model_weights.npy')
ans_y = np.dot(normed_test_x, test_w)

In [0]:
df = pd.DataFrame(np.empty((240, 2)), index = np.arange(240) + 1, columns = ['id', 'value'])
for i in range(240):
  df.iloc[i, 0] = 'id_' + str(i)
  df.iloc[i,1] = ans_y[i][0]
df.to_csv('submission.csv', index = False)